In [1]:
import pandas as pd 
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
data = pd.read_csv("/content/drive/MyDrive/preprocessed_data.csv")
data = data.dropna()


In [72]:
sampled_data = data.sample(n=5000, random_state=42)

In [73]:
sampled_data.head()

,Unnamed: 0,text,dialect
25652,25652,يمكن تستغرب شء,2
52025,52025,خلصتي ولسه بتسقطي دي هوايه عندك بقي,0
5368,5368,صارت معايا مره,2
37970,37970,ومال تماره راه اغلب كانو الرباط شرو تماره بحكم...,3
117693,117693,الله يلعن السياسه يللي بتبعد الشرفاء بالبلد هي...,1


In [41]:
data["dialect"].shape

(147642,)

## WORD2VEC

1: using tfidf

In [17]:
## word2vec 
from gensim.models import Word2Vec


## MODEL 

In [3]:
# SPLIT DATA
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data["text"], data['dialect'], test_size=0.2, random_state=42)


In [92]:
#                                            svc 
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

model = SVC(kernel='linear', C=1, gamma='auto')

pipeline = Pipeline([
    ('tfidf',  TfidfVectorizer()),
    ('svc', model)
])
pipeline.fit(train_X, train_y)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('svc', SVC(C=1, gamma='auto', kernel='linear'))])

In [93]:
# evaluation
from sklearn.metrics import classification_report
predictions = pipeline.predict(test_X)

In [94]:
print(classification_report(test_y, predictions))


              precision    recall  f1-score   support

           0       0.82      0.92      0.86     11525
           1       0.86      0.82      0.84      5607
           2       0.78      0.82      0.80      7251
           3       0.90      0.64      0.75      2298
           4       0.85      0.57      0.68      2848

    accuracy                           0.82     29529
   macro avg       0.84      0.75      0.79     29529
weighted avg       0.82      0.82      0.82     29529



## evaluation on 5000 samples

In [85]:
print(classification_report(test_y, predictions))


              precision    recall  f1-score   support

           0       0.73      0.90      0.80       398
           1       0.78      0.66      0.72       182
           2       0.64      0.72      0.68       244
           3       0.79      0.45      0.57        85
           4       0.86      0.33      0.48        91

    accuracy                           0.72      1000
   macro avg       0.76      0.61      0.65      1000
weighted avg       0.73      0.72      0.71      1000



In [121]:
pipeline.predict(["الله يلعن السياسه يللي بتبعد الشرفاء بالبلد هي...	"])

array([1])

In [5]:
import pickle
pickle.dump(pipeline ,open("/content/drive/MyDrive/SVC_model.pkl","wb"))


In [9]:
model_path = "/content/drive/MyDrive/SVC_model.pkl"

In [10]:
def dialect_predict(model_path,sentence):
  model = pickle.load(open(model_path,"rb"))
  sentence =[sentence]
  dialect = model.predict(sentence)

  if dialect == 0:
    country = "Egypt"

  elif  dialect == 1 :
    country = "Lebnanon"

  elif dialect ==   2 :
    country = "Libya"

  elif dialect == 3:
    country = "Morocco"

  else:
    country = "Sudan"
  
  return country    


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
dialect_predict(model_path,"ماما زمانها جايه")

'Egypt'